In [1]:
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [2]:
df = pd.read_csv('../Output_Folder/Santa_Barbara_Yelps.csv')

In [3]:
df

,business_id,name,address,city,state,postal_code,review_count,review_id,stars,text
0,gebiRewfieSdtt17PTW6Zg,Hibachi Steak House & Sushi Bar,502 State St,Santa Barbara,CA,93101,488,pUycOfUwM8vqX7KjRRhUEA,3.0,Had a party of 6 here for hibachi. Our waitres...
1,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,389,L0jv8c2FbpWSlfNC6bbUEA,5.0,What a great addition to the Funk Zone! Grab ...
2,qO9dNNIvNbCBd8ZgjxMxgQ,HOME Santa Barbara,14 Parker Way,Santa Barbara,CA,93101,13,2u5Skti5mZam_-XTKPelvA,5.0,"Farmhouse, rustic, chic.Helpful staff with gre..."
3,SZU9c8V2GuREDN5KgyHFJw,Santa Barbara Shellfish Company,230 Stearns Wharf,Santa Barbara,CA,93101,2404,4zopEEPqfwm-c_FNpeHZYw,5.0,We were a bit weary about trying the Shellfish...
4,YbnJYHNp_fHbI-hcFg48vQ,Santa Barbara Adventure Company,32 E Haley St,Santa Barbara,CA,93101,195,HuWEnZr7-0HveaqXxyywgg,5.0,The kayaking tour at the Santa Cruz Island was...
...,...,...,...,...,...,...,...,...,...,...
481,Vz1QSF4nurQv2-dok55jZg,"Howard Gross, MD","1722 State St, Ste 201",Santa Barbara,CA,93101,16,NaN,NaN,NaN
482,jNWePdDkWzwt4oQ5UZulIg,Faitell Attractions,619 State St,Santa Barbara,CA,93101,10,NaN,NaN,NaN
483,8-ZExybRuyyXZf6aESgc3g,Paseo Nuevo Shopping Center,651 Paseo Nuevo,Santa Barbara,CA,93101,125,NaN,NaN,NaN
484,EHF8hnwsWES8lmedq5eqqA,Skivvies Esthetics,"11 W. de La Guerra St, Ste B",Santa Barbara,CA,93101,10,NaN,NaN,NaN


In [4]:
df2 = pd.read_csv('../Final_Business.csv')
df2.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6
4,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13


In [5]:
df2.dtypes

business_id      object
name             object
address          object
city             object
state            object
postal_code      object
latitude        float64
longitude       float64
stars           float64
review_count      int64
dtype: object

In [6]:
df2.shape

(94976, 10)

In [7]:
merged_df = df2.merge(df, how = 'left', on = ['business_id'])

In [8]:
merged_df

,business_id,name_x,address_x,city_x,state_x,postal_code_x,latitude,longitude,stars_x,review_count_x,name_y,address_y,city_y,state_y,postal_code_y,review_count_y,review_id,stars_y,text
0,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,MO,63144,38.627695,-90.340465,2.5,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95299,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95300,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95301,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95302,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
merged_df.dtypes

business_id        object
name_x             object
address_x          object
city_x             object
state_x            object
postal_code_x      object
latitude          float64
longitude         float64
stars_x           float64
review_count_x      int64
name_y             object
address_y          object
city_y             object
state_y            object
postal_code_y     float64
review_count_y    float64
review_id          object
stars_y           float64
text               object
dtype: object

In [10]:
merged_df.isna().sum()

business_id           0
name_x                0
address_x          3218
city_x                0
state_x               0
postal_code_x         6
latitude              0
longitude             0
stars_x               0
review_count_x        0
name_y            94818
address_y         94830
city_y            94818
state_y           94818
postal_code_y     94818
review_count_y    94818
review_id         94919
stars_y           94919
text              94919
dtype: int64

In [11]:
S_B_Data =( merged_df[merged_df['postal_code_x'] == '93101'])

In [12]:
S_B_Data

,business_id,name_x,address_x,city_x,state_x,postal_code_x,latitude,longitude,stars_x,review_count_x,name_y,address_y,city_y,state_y,postal_code_y,review_count_y,review_id,stars_y,text
17,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,H&M,827-833 State St,Santa Barbara,CA,93101.0,24.0,nD9ug6O1rQQ_B6OuY5eitw,5.0,The people at the store amazing! We came in to...
18,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,H&M,827-833 State St,Santa Barbara,CA,93101.0,24.0,ok15zpwBjImmf3KlSGUkJw,4.0,This is the best priced place for tank tops th...
19,noByYNtDLQAra9ccqxdfDw,H&M,827-833 State St,Santa Barbara,CA,93101,34.420209,-119.700460,3.0,24,H&M,827-833 State St,Santa Barbara,CA,93101.0,24.0,Q3LfxYXMB_odT6HJRQdqkQ,4.0,Glad to have H&M in Santa Barbara. What an exc...
48,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,389.0,L0jv8c2FbpWSlfNC6bbUEA,5.0,What a great addition to the Funk Zone! Grab ...
49,IDtLPgUrqorrpqSLdfMhZQ,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101,34.414445,-119.690672,4.0,389,Helena Avenue Bakery,"131 Anacapa St, Ste C",Santa Barbara,CA,93101.0,389.0,LFpaQzYkP5Pzm5lEjJpTRw,5.0,Possibly the best breakfast sandwich EVER. On...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94765,bVaRZDHkWdsHuARGxxpREw,Sushi Bar 29,1134 Chapala St,Santa Barbara,CA,93101,34.422291,-119.705339,4.5,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94795,US1LvQeJ03KH0sViR-KfVQ,Truth Aquatics,301 W Cabrillo Blvd,Santa Barbara,CA,93101,34.408290,-119.691469,4.5,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95198,izSgTrqebu8bN8ONOCs6cQ,Oat Bakery,5 W Haley St,Santa Barbara,CA,93101,34.416548,-119.695626,5.0,123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95233,IRBhPAC4ZoDpXazpoB3epQ,Good Stuff Baked Treats,NaN,Santa Barbara,CA,93101,34.420334,-119.710749,5.0,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
S_B_Data.shape

(1411, 19)

In [14]:
S_B_Data.isna().sum()

business_id          0
name_x               0
address_x          109
city_x               0
state_x              0
postal_code_x        0
latitude             0
longitude            0
stars_x              0
review_count_x       0
name_y             925
address_y          937
city_y             925
state_y            925
postal_code_y      925
review_count_y     925
review_id         1026
stars_y           1026
text              1026
dtype: int64

In [15]:


S_B_Data.to_csv('../Output_Folder/S_B_Data.csv', index=False)

In [17]:
S_B_Data.to_excel('../Output_Folder/S_B_Data.xlsx')